In [1]:
import numpy as np
import pandas as pd
from scipy import linalg as la
from matplotlib import pyplot as plt
import cmath

# Problem 1

In [70]:
def isdrazin(A,k,Ad):
    return np.all([[np.dot(A,Ad)==np.dot(Ad,A)], [np.dot(np.linalg.matrix_power(A,k+1),Ad)==np.linalg.matrix_power(A,k)], [np.dot(np.dot(Ad,A),Ad)==Ad]])
A=np.array([[1,3,0,0],[0,1,3,0],[0,0,1,3],[0,0,0,0]])
Ad=np.array([[1,-3,9,81],[0,1,-3,-18],[0,0,1,3],[0,0,0,0]])
B=np.array([[1,1,3],[5,2,6],[-2,-1,-3]])
Bd=np.zeros([3,3])
isdrazin(A,1,Ad), isdrazin(B,3,Bd)
                                    

(True, True)

# Problem 2

In [75]:
def drazin(A,tol=1e-9):
    n=len(A[0,:])
    f=lambda x:abs(x)>tol
    g=lambda x: abs(x)<=tol
    Q1,S,k1=la.schur(A, sort=f)
    Q2,T,k2=la.schur(A, sort=g)
    U=np.column_stack([S[:,:k1],T[:,:(n-k1)]])
    Uinv=np.linalg.inv(U)
    V=np.dot(np.dot(Uinv,A),U)
    Z=np.zeros([n,n])
    if k1!=0:
        Minv=np.linalg.inv(V[:(k1),:(k1)])
        Z[:(k1),:(k1)]=Minv
    return np.dot(np.dot(U,Z),Uinv)
drazin(A,1e-16)


array([[  1.,  -3.,   9.,  81.],
       [  0.,   1.,  -3., -18.],
       [  0.,   0.,   1.,   3.],
       [  0.,   0.,   0.,   0.]])

# Problem 3

In [144]:
def eff_res(A):
    n=len(A[0,:])
    L=np.empty_like(A)
    for i in range(n):
        for j in range(n):
            if j==i:
                L[i,j]=sum(A[i,:])
            if j!=i:
                L[i,j]=-A[i,j]
    I=np.eye(n)
    R=np.copy(L).astype(float)
    for j in range(n):
        L0=np.copy(L)
        L0[j,:]=I[j,:]
        L0d=drazin(L0)
        R[:,j]=np.diag(L0d)
        R[j,j]=0
    return R
C=np.array([[0,1,0,0],[1,0,1,0],[0,1,0,1],[0,0,1,0]])
eff_res(C)

array([[ 0.,  1.,  2.,  3.],
       [ 1.,  0.,  1.,  2.],
       [ 2.,  1.,  0.,  1.],
       [ 3.,  2.,  1.,  0.]])

# Problem 4

In [301]:
class LinkPredictor(object):
    def __init__(self, name):
        self.name=name
        data=np.array(pd.read_csv(self.name))
        self.namelist=sorted(list(set(data[:,0]).union(set(data[:,1]))))
        A=np.random.randn(len(self.namelist),len(self.namelist))
        n=len(self.namelist)
        name1=np.random.randn(len(data[:,0]),n)
        name2=np.random.randn(len(data[:,1]),n)
        for i in range(n):
            name1[:,i]=data[:,0]==self.namelist[i]
            name2[:,i]=data[:,1]==self.namelist[i]
        for i in range(n):
            for j in range(n):
                A[i,j]=np.dot(name1[:,i],name2[:,j])+np.dot(name1[:,j],name2[:,i])
        self.adjmatrix=A
        self.eff_res=eff_res(A)
        
        

# Problem 5

In [395]:
class LinkPredictor(object):
    def __init__(self, name):
        self.name=name
        data=np.array(pd.read_csv(self.name))
        self.namelist=sorted(list(set(data[:,0]).union(set(data[:,1]))))
        A=np.random.randn(len(self.namelist),len(self.namelist))
        n=len(self.namelist)
        name1=np.random.randn(len(data[:,0]),n)
        name2=np.random.randn(len(data[:,1]),n)
        for i in range(n):
            name1[:,i]=data[:,0]==self.namelist[i]
            name2[:,i]=data[:,1]==self.namelist[i]
        for i in range(n):
            for j in range(n):
                A[i,j]=np.dot(name1[:,i],name2[:,j])+np.dot(name1[:,j],name2[:,i])
        self.adjmatrix=A
        self.eff_res=eff_res(A)
    def predict_link(self, node):
        if node==None:
            A=self.adjmatrix
            D=self.eff_res
            mask=D==0
            D[mask]=np.max(D)+1
            D[A==1]=np.max(D)+1
            link=D.argmin()
            return namelist[link//len(self.namelist)],namelist[link%len(self.namelist)]
        if node in namelist:
            A=self.adjmatrix
            D=self.eff_res
            mask=D==0
            D[mask]=np.max(D)+1
            D[A==1]=np.max(D)+1
            i=namelist.index(node)
            link=np.argmin(D[i,:])
            return namelist[link]
        else:
            raise ValueError("Node has to be None or in name list.")
    def add_link(self, node1, node2):
        if np.all([[node1 in namelist,node2 in namelist]]):
            i=namelist.index(node1)
            j=namelist.index(node2)
            A=self.adjmatrix
            A[i,j]=1
            A[j,i]=1
            self.adjmatrix=A
        else:
            raise ValueError("The two nodes have to be in the network.")
soc=LinkPredictor('social_network.csv')
soc.predict_link(None), soc.predict_link('Melanie'), soc.predict_link('Alan')



(('Oliver', 'Emily'), 'Carol', 'Sonia')